In [1]:
%matplotlib inline
import sys
import cv2
from matplotlib import pyplot as plt
import numpy as np
import aruco

In [2]:
# Globals
CV_CAP_PROP_POS_MSEC = 0
CV_CAP_PROP_FRAME_COUNT = 7

def dist(x1,y1,x2,y2):
    return ((x2-x1)**2 + (y2-y1)**2)**0.5

In [3]:
# set up - choose files
stripped_raw_data_file_path = "board_stripped.txt"
video_file_path = "AlisonBoard1-1-recording.avi"
output_file_path = "alison_test_output.csv"

In [4]:
# create detector
detector = aruco.MarkerDetector()

In [5]:
# load eye tracking data
eye_data = []
data_file_object  = open(stripped_raw_data_file_path, "r")
data_file_object.readline()
for line in data_file_object.readlines():
    eye_data.append(map(float, line.split()))
data_file_object.close()

In [6]:
# load video

lastStatus =  "IN" # was the last record IN or OUT?
currentSessionFrameCount = 0 # to time sessions
sessionMinimum = 3 # minimum frame count to count as session (for smoothing)
sessions = []

cap = cv2.VideoCapture(video_file_path)
ret, frame = cap.read()
    
if not ret:
    print("can't open video!")
    sys.exit(-1)
    
time = cap.get(CV_CAP_PROP_POS_MSEC)


while ret and int(time/16.3) < len(eye_data):
    
    markers = detector.detect(frame)

    if len(markers) == 0:
        if lastStatus == "OUT":
            currentSessionFrameCount += 1
        else:
            sessions.append(["IN", currentSessionFrameCount])
            currentSessionFrameCount = 1
        lastStatus = "OUT"
    else:
        if lastStatus == "IN":
            currentSessionFrameCount += 1
        else:
            sessions.append(["OUT", currentSessionFrameCount])
            currentSessionFrameCount = 1
        lastStatus = "IN"
    
    # read next frame
    ret, frame = cap.read()
    time = cap.get(CV_CAP_PROP_POS_MSEC)

if currentSessionFrameCount:
    if lastStatus == "OUT":
        sessions.append(["OUT", currentSessionFrameCount])
    else:
        sessions.append(["IN", currentSessionFrameCount])

i=0
while i < len(sessions):
    if sessions[i][1] < sessionMinimum:
        sessions[i-1][1] += sessions[i][1]
        del sessions[i]
        while i < len(sessions) and sessions[i][0] == sessions[i-1][0]:
            sessions[i-1][1] += sessions[i][1]
            del sessions[i]
    else:
        i += 1

In [7]:
# print visualization by frame

print "Visualization by frame, divided by second:"
printedframes = 0
for session in sessions:
    for _ in xrange(session[1]):
        if session[0] == "IN":
            if printedframes%24 == 0:
                print
                print printedframes/24 + 1,
            print "X",
            printedframes += 1
        elif session[0] == "OUT":
            if printedframes%24 == 0:
                print
                print printedframes/24 + 1,
            print "-",
            printedframes += 1

Visualization by frame, divided by second:

1 X X X X X X X X X X X X X X X X X X X X X X X X
2 X X X X X X X X X X X X X X X X X X X X X X X X
3 X X X X X X X X X X X X X X X X X X X X X X X X
4 X X X X X X X X X X X X X X X X X X X X X X X X
5 X X X X X X X X X X X X X X X X X X X X X X X X
6 X X X X X X X X X X X X X X X X X X X X X X X X
7 X X X X X X X X X X X X X X X X X X X X X X X X
8 X X X X X X X X X X X X X X X X X X X X X X X X
9 X X X X X X X X X X X X X X X X X X X X X X X X
10 X X X X X X X X X X X X X X X X X X X X X X X X
11 X X X X X X X X X X X X X X X X X X X X X X X X
12 X X X X X X X X X X X X X X X X X X X X X X X X
13 X X X X X X X X X X X X X X X X X X X X X X X X
14 X X X X X X X X X X X X X X X X X X X X X X X X
15 X - - - - - - - - - - - - - - - - - - - X X X X
16 X X X X X X X X X X X X X X X X X X X X X X X X
17 X X X X X X X X X X X X X X X X X X X X X X X X
18 X X X X X X X X X X X X X X X X X X X X X X X X
19 X X X X X X X X X X X X X X X X X X X X X X 